In [1]:
import gensim
import pandas as pd
# import keras
import pickle
import numpy as np
import tensorflow as tf

2024-04-25 18:10:12.574092: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
filename='preprocessed_data.pkl'

In [3]:
data=pickle.load(open(filename,'rb')) ###open preprocessed data

In [4]:
len(data)

3243

In [5]:
Y= data.iloc[:,1].values
len(Y)

3243

In [6]:
###Manually resolve the tabbed data errors

index = [791,  792,  793,  830,  831,  832,  833,  839,  844, 846, 852,  853,
         855,  858,  860,  876,  882,  890,  892,  904,  909,  912,  936,
         938,  943,  945,  957,  964,  965,  967,  969,  972,  975,  976,
        1000, 1007, 1013, 1018, 1297] 
for i in index:
    Y[i] = 0  ###replace value of selected indices of AD column with 0

idx = [1072, 1101, 1184, 1244]
for i in idx:
    Y[i] = 1  ###replace value of selected indices of AD column with 1
Y = Y.astype(int)

In [7]:
X = data['Transcript'].tolist()

In [8]:
from sklearn.model_selection import train_test_split

In [19]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 0) ##split the data in training and testing
X_train[10]

'little girl goin g'

In [10]:
print(len(X_train)) ###length of training, testing data
print(len(X_test))
print(len(Y_train))
print(len(Y_test))

2594
649
2594
649


In [11]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [12]:
def create_tokens(lines):
    tokens = Tokenizer()
    
    ###function to create tokens
    tokens.fit_on_texts(lines)
    return tokens

In [13]:
train_tokenizer = create_tokens(X_train)


In [14]:
import json

# Save Tokenizer configuration as a JSON string
tokenizer_json = train_tokenizer.to_json()

# Save Tokenizer JSON string to a file
with open('tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(tokenizer_json)


In [15]:
def encode_sentences(tokens,length,lines):   ###function to pad the data to maximum phrase length
    X=tokens.texts_to_sequences(lines)
    X=pad_sequences(X,length,padding='post')
    return X    

In [16]:
X_train = encode_sentences(train_tokenizer,100, X_train)
X_test = encode_sentences(train_tokenizer,100, X_test)
X_train[0]


array([  1,  73, 167,  20,   4,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0], dtype=int32)

In [90]:
print(train_tokenizer)

In [43]:
print(X_train.shape)   ###check for dimensions
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(2594, 100)
(649, 100)
(2594,)
(649,)


In [84]:
print(train_tokenizer.word_index) ###print the word index
vocab_len= len(train_tokenizer.word_index ) + 1 
vocab_len   ###vocab length

{'and': 1, 'uh': 2, 'it': 3, 'cookie': 4, 'g': 5, 'dish': 6, 'water': 7, 'jar': 8, 'mother': 9, 'stool': 10, 'boy': 11, 'little': 12, 'girl': 13, 'sink': 14, 'see': 15, 'window': 16, 'um': 17, 'cooky': 18, 'floor': 19, 'reaching': 20, 'look': 21, 'like': 22, 'running': 23, 'standing': 24, 'drying': 25, 'open': 26, 'fall': 27, 'cup': 28, 'getting': 29, 'laugh': 30, 'overflowing': 31, 'one': 32, 'oh': 33, 'kitchen': 34, 'hand': 35, 'going': 36, 'falling': 37, 'get': 38, 'well': 39, 'gonna': 40, 'two': 41, 'curtain': 42, 'know': 43, 'plate': 44, 'washing': 45, 'xxx': 46, 'think': 47, 'kid': 48, 'looking': 49, 'counter': 50, 'outside': 51, 'sister': 52, 'door': 53, 'f': 54, 'let': 55, 'r': 56, 'want': 57, 'anything': 58, 'onto': 59, 'child': 60, 'foot': 61, 'house': 62, 'lady': 63, 'guess': 64, 'something': 65, 'okay': 66, 'tree': 67, 'else': 68, 'cupboard': 69, 'thing': 70, 'maybe': 71, 'tell': 72, 'also': 73, 'saying': 74, 'right': 75, 'cabinet': 76, 'lid': 77, 'three': 78, 'exc': 79, 's

847

# Attention MODEL - Case 1: Training the model using randomly initialized embedding

In [70]:
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import LSTM, Bidirectional, Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Flatten
from keras.layers import Dense 
from keras.layers import Dropout
from keras.callbacks import ModelCheckpoint
# from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier

In [71]:
from keras.layers import Layer
import keras.backend as K
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()


In [72]:

seq_input = tf.keras.Input(shape=(100,))
e = Embedding(vocab_len, 100, input_length= 100)(seq_input)
conv1 = Conv1D(filters=128,kernel_size=5,activation='relu',strides= 1, kernel_initializer='he_uniform')(e)
pool = MaxPooling1D(pool_size=2)(conv1)
dropout= Dropout(0.5)(pool)
lstm1 = Bidirectional(LSTM(10, return_sequences=True))(dropout)
att=attention()(lstm1)
dense1 = Dense(units = 20, activation='relu', kernel_initializer='he_uniform')(att)
output= Dense(units = 1, activation = 'sigmoid', kernel_initializer= 'glorot_uniform')(dense1)
model=tf.keras.Model(seq_input,output)


In [73]:
model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [74]:
attention_history=model.fit(X_train, Y_train, epochs = 25, batch_size = 64, verbose = 1, validation_split = 0.1)

Epoch 1/25


KeyboardInterrupt: 

In [75]:
score = model.evaluate(X_test,Y_test,verbose = 1)
score[1]

21/21 [==============================] - 2s 7ms/step - loss: 0.6949 - accuracy: 0.4730


0.47303542494773865

In [76]:
def model_attention_create(optimizer = 'Adam',
                 dropout_rate = 0.2, nb_filters = 256, kernel_size = 5,pool_size=2, units = 128,neurons = 20):
    
    seq_input = tf.keras.Input(shape=(100,))
    e = Embedding(vocab_len, 100, input_length= 100)(seq_input)
    conv1 = Conv1D(nb_filters,kernel_size,activation='relu',strides= 1, kernel_initializer='he_uniform')(e)
    pool = MaxPooling1D(pool_size)(conv1)
    dropout= Dropout(0.5)(pool)
    lstm1 = Bidirectional(LSTM(10, return_sequences=True))(dropout)
    att=attention()(lstm1)
    dense1 = Dense(units = neurons, activation='relu', kernel_initializer='he_uniform')(att)
    output= Dense(units = 1, activation = 'sigmoid', kernel_initializer= 'glorot_uniform')(dense1)
    model=tf.keras.Model(seq_input,output) 
    model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [81]:
param_grid = {
    'units': [10,20,64,128]
    # 'units': [10]

}                                                             ####perform grid search to find the hyperparameters

model = KerasClassifier(build_fn = model_attention_create, verbose = 2,units = [10,20,64,128])
print(model.get_params().keys())
grid_search = GridSearchCV(estimator = model, param_grid = param_grid, verbose = 2)
grid_results = grid_search.fit(X_train, Y_train, epochs = 10)
print('Best accuracy: ', grid_results.best_score_)
print('Best parameters: ', grid_results.best_params_)

dict_keys(['model', 'build_fn', 'warm_start', 'random_state', 'optimizer', 'loss', 'metrics', 'batch_size', 'validation_batch_size', 'verbose', 'callbacks', 'validation_split', 'shuffle', 'run_eagerly', 'epochs', 'units', 'class_weight'])
Fitting 5 folds for each of 4 candidates, totalling 20 fits


/Users/mac/opt/anaconda3/envs/audenv称/lib/python3.8/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/10
65/65 - 8s - loss: 0.6929 - accuracy: 0.5118 - 8s/epoch - 116ms/step
Epoch 2/10
65/65 - 3s - loss: 0.6892 - accuracy: 0.5436 - 3s/epoch - 39ms/step
Epoch 3/10
65/65 - 3s - loss: 0.6204 - accuracy: 0.6843 - 3s/epoch - 40ms/step
Epoch 4/10
65/65 - 3s - loss: 0.5497 - accuracy: 0.7118 - 3s/epoch - 39ms/step
Epoch 5/10
65/65 - 3s - loss: 0.4954 - accuracy: 0.7812 - 3s/epoch - 39ms/step
Epoch 6/10
65/65 - 3s - loss: 0.4557 - accuracy: 0.7937 - 3s/epoch - 39ms/step
Epoch 7/10
65/65 - 3s - loss: 0.5052 - accuracy: 0.7624 - 3s/epoch - 39ms/step
Epoch 8/10
65/65 - 3s - loss: 0.4194 - accuracy: 0.8241 - 3s/epoch - 39ms/step
Epoch 9/10
65/65 - 3s - loss: 0.3719 - accuracy: 0.8414 - 3s/epoch - 39ms/step
Epoch 10/10
65/65 - 3s - loss: 0.3428 - accuracy: 0.8530 - 3s/epoch - 40ms/step
17/17 - 1s - 988ms/epoch - 58ms/step
[CV] END ...........................................units=10; total time=  32.5s


/Users/mac/opt/anaconda3/envs/audenv称/lib/python3.8/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/10
65/65 - 7s - loss: 0.6931 - accuracy: 0.5022 - 7s/epoch - 115ms/step
Epoch 2/10
65/65 - 3s - loss: 0.6863 - accuracy: 0.5494 - 3s/epoch - 43ms/step
Epoch 3/10
65/65 - 3s - loss: 0.6055 - accuracy: 0.6945 - 3s/epoch - 39ms/step
Epoch 4/10
65/65 - 3s - loss: 0.5168 - accuracy: 0.7730 - 3s/epoch - 39ms/step
Epoch 5/10
65/65 - 3s - loss: 0.4319 - accuracy: 0.8289 - 3s/epoch - 39ms/step
Epoch 6/10
65/65 - 3s - loss: 0.3711 - accuracy: 0.8602 - 3s/epoch - 39ms/step
Epoch 7/10
65/65 - 3s - loss: 0.3190 - accuracy: 0.8819 - 3s/epoch - 39ms/step
Epoch 8/10
65/65 - 3s - loss: 0.2834 - accuracy: 0.8930 - 3s/epoch - 39ms/step
Epoch 9/10
65/65 - 3s - loss: 0.2502 - accuracy: 0.9089 - 3s/epoch - 40ms/step
Epoch 10/10
65/65 - 3s - loss: 0.2349 - accuracy: 0.9147 - 3s/epoch - 39ms/step
17/17 - 1s - 969ms/epoch - 57ms/step
[CV] END ...........................................units=10; total time=  32.2s


...

Epoch 1/10
65/65 - 7s - loss: 0.6931 - accuracy: 0.5166 - 7s/epoch - 115ms/step
Epoch 2/10
65/65 - 3s - loss: 0.6842 - accuracy: 0.5528 - 3s/epoch - 41ms/step
Epoch 3/10
65/65 - 3s - loss: 0.6092 - accuracy: 0.6853 - 3s/epoch - 41ms/step
Epoch 4/10
65/65 - 3s - loss: 0.4996 - accuracy: 0.7812 - 3s/epoch - 42ms/step
Epoch 5/10
65/65 - 3s - loss: 0.4166 - accuracy: 0.8246 - 3s/epoch - 50ms/step
Epoch 6/10
65/65 - 3s - loss: 0.3424 - accuracy: 0.8607 - 3s/epoch - 45ms/step
Epoch 7/10
65/65 - 3s - loss: 0.2981 - accuracy: 0.8834 - 3s/epoch - 43ms/step
Epoch 8/10
65/65 - 3s - loss: 0.2685 - accuracy: 0.8978 - 3s/epoch - 46ms/step
Epoch 9/10
65/65 - 3s - loss: 0.2493 - accuracy: 0.9036 - 3s/epoch - 40ms/step
Epoch 10/10
65/65 - 3s - loss: 0.2107 - accuracy: 0.9210 - 3s/epoch - 41ms/step
17/17 - 1s - 984ms/epoch - 58ms/step
[CV] END ..........................................units=128; total time=  34.5s


/Users/mac/opt/anaconda3/envs/audenv称/lib/python3.8/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/10
65/65 - 8s - loss: 0.6942 - accuracy: 0.5096 - 8s/epoch - 123ms/step
Epoch 2/10
65/65 - 3s - loss: 0.6848 - accuracy: 0.5559 - 3s/epoch - 41ms/step
Epoch 3/10
65/65 - 3s - loss: 0.6009 - accuracy: 0.6961 - 3s/epoch - 39ms/step
Epoch 4/10
65/65 - 3s - loss: 0.5030 - accuracy: 0.7669 - 3s/epoch - 39ms/step
Epoch 5/10
65/65 - 3s - loss: 0.4392 - accuracy: 0.8107 - 3s/epoch - 39ms/step
Epoch 6/10
65/65 - 3s - loss: 0.3916 - accuracy: 0.8377 - 3s/epoch - 39ms/step
Epoch 7/10
65/65 - 3s - loss: 0.3288 - accuracy: 0.8695 - 3s/epoch - 39ms/step
Epoch 8/10
65/65 - 3s - loss: 0.2939 - accuracy: 0.8834 - 3s/epoch - 39ms/step
Epoch 9/10
65/65 - 3s - loss: 0.2440 - accuracy: 0.9114 - 3s/epoch - 42ms/step
Epoch 10/10
65/65 - 3s - loss: 0.2353 - accuracy: 0.9099 - 3s/epoch - 41ms/step
17/17 - 1s - 977ms/epoch - 57ms/step
[CV] END ..........................................units=128; total time=  33.0s


/Users/mac/opt/anaconda3/envs/audenv称/lib/python3.8/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/10
82/82 - 8s - loss: 0.6936 - accuracy: 0.5112 - 8s/epoch - 95ms/step
Epoch 2/10
82/82 - 3s - loss: 0.6637 - accuracy: 0.5875 - 3s/epoch - 40ms/step
Epoch 3/10
82/82 - 3s - loss: 0.5452 - accuracy: 0.7394 - 3s/epoch - 39ms/step
Epoch 4/10
82/82 - 3s - loss: 0.4530 - accuracy: 0.8069 - 3s/epoch - 39ms/step
Epoch 5/10
82/82 - 3s - loss: 0.3932 - accuracy: 0.8389 - 3s/epoch - 39ms/step
Epoch 6/10
82/82 - 3s - loss: 0.3543 - accuracy: 0.8608 - 3s/epoch - 39ms/step
Epoch 7/10
82/82 - 3s - loss: 0.3020 - accuracy: 0.8859 - 3s/epoch - 40ms/step
Epoch 8/10
82/82 - 3s - loss: 0.2659 - accuracy: 0.9017 - 3s/epoch - 39ms/step
Epoch 9/10
82/82 - 3s - loss: 0.2489 - accuracy: 0.9056 - 3s/epoch - 39ms/step
Epoch 10/10
82/82 - 3s - loss: 0.2215 - accuracy: 0.9156 - 3s/epoch - 39ms/step
Best accuracy:  0.804167503589469
Best parameters:  {'units': 128}


In [79]:
param_grid = {
    'nb_filters': [128,256],
    'kernel_size':[3,5],
    'pool_size':[2,4]

}                                                             ####perform grid search to find the hyperparameters

model = KerasClassifier(build_fn = model_attention_create, verbose = 2)
grid_search = GridSearchCV(estimator = model, param_grid = param_grid, verbose = 2)
grid_results = grid_search.fit(X_train, Y_train, epochs = 10)
print('Best accuracy: ', grid_results.best_score_)
print('Best parameters: ', grid_results.best_params_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


ValueError: Invalid parameter kernel_size for estimator KerasClassifier.
This issue can likely be resolved by setting this parameter in the KerasClassifier constructor:
`KerasClassifier(kernel_size=3)`
Check the list of available parameters with `estimator.get_params().keys()`

In [ ]:
param_grid = {
    'neurons':[10,20,64,128]
}                                                             ####perform grid search to find the hyperparameters

model = KerasClassifier(build_fn = model_attention_create, verbose = 2)
grid_search = GridSearchCV(estimator = model, param_grid = param_grid, verbose = 2)
grid_results = grid_search.fit(X_train, Y_train, epochs = 10)
print('Best accuracy: ', grid_results.best_score_)
print('Best parameters: ', grid_results.best_params_)

In [ ]:
param_grid = {
    'dropout_rate':[0.1,0.2,0.3,0.4,0.5]
}                                                             ####perform grid search to find the hyperparameters

model = KerasClassifier(build_fn = model_attention_create, verbose = 2)
grid_search = GridSearchCV(estimator = model, param_grid = param_grid, verbose = 2)
grid_results = grid_search.fit(X_train, Y_train, epochs = 10)
print('Best accuracy: ', grid_results.best_score_)
print('Best parameters: ', grid_results.best_params_)

In [ ]:
param_grid = {
    'batch_size':[32,64,128,256]
}                                                             ####perform grid search to find the hyperparameters

model = KerasClassifier(build_fn = model_attention_create, verbose = 2)
grid_search = GridSearchCV(estimator = model, param_grid = param_grid, verbose = 2, cv=2)
grid_results = grid_search.fit(X_train, Y_train, epochs = 10)
print('Best accuracy: ', grid_results.best_score_)
print('Best parameters: ', grid_results.best_params_)

# MODEL with Optimised Parameters

In [ ]:
seq_input = keras.Input(shape=(100,))
e = Embedding(vocab_len, 100, input_length= 100)(seq_input)
conv1 = Conv1D(filters=256,kernel_size=5,activation='relu',strides= 1, kernel_initializer='he_uniform')(e)
pool = MaxPooling1D(pool_size=4)(conv1)
dropout= Dropout(0.5)(pool)
lstm1 = Bidirectional(LSTM(64, return_sequences=True))(dropout)
att=attention()(lstm1)
dense1 = Dense(units = 128, activation='relu', kernel_initializer='he_uniform')(att)
output= Dense(units = 1, activation = 'sigmoid', kernel_initializer= 'glorot_uniform')(dense1)
att_model=keras.Model(seq_input,output)


In [ ]:
att_model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
attention_history=att_model.fit(X_train, Y_train, epochs = 50, batch_size = 32, verbose = 1, validation_split = 0.1)

In [ ]:
score = att_model.evaluate(X_test,Y_test,verbose = 1)
score[1]

In [ ]:
import matplotlib.pyplot as plt                             ###plot loss on train and val data
plt.plot(attention_history.history['loss'])
plt.plot(attention_history.history['val_loss'])
plt.title('Model Loss Plot')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Train', 'Test'],loc = 'upper right')
plt.show()

In [ ]:
att_model.save('attention_model.hdf5')

In [ ]:
att_model.load_weights('attention_model.hdf5')

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, roc_curve, roc_auc_score
import matplotlib.pyplot as plt

In [ ]:
Y_pred = att_model.predict(X_test)
Y_pred = (Y_pred >0.5)

In [ ]:
confusion = confusion_matrix(Y_test,Y_pred)        ###Model evaluation
precision = precision_score(Y_test,Y_pred)
recall = recall_score(Y_test,Y_pred)
F1 = f1_score(Y_test,Y_pred)
specificity = (confusion[0][0]/(confusion[0][0]+confusion[0][1]))
print('confusion matrix {}'.format(confusion))
print('precision is {}'.format(precision))
print('recall is  {}'.format(recall))
print('F1 score {}'.format(F1))
print('specificity is {}'.format(specificity))

In [ ]:
Y_pred_proba = att_model.predict(X_test)
fpr, tpr, _ = roc_curve(Y_test, Y_pred_proba)
auc = roc_auc_score(Y_test, Y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()

# Attention MODEL: Case 2 - Using Pretrained embedding

In [ ]:
embeddings = dict()                                 ##read glove vector file
f = open('../glove.6B.100d.txt',encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    vectors = np.asarray(values[1:],dtype='float32')
    embeddings[word]=vectors
f.close()

In [ ]:
embedding_matrix = np.zeros((vocab_len, 100))      ###create a matrix consist of words and its vectors
for word, i in train_tokenizer.word_index.items():
    embedding_vector = embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
seq_input = keras.Input(shape=(100,))
e = Embedding(vocab_len, 100, input_length= 100,weights = [embedding_matrix], trainable= False)(seq_input)
conv1 = Conv1D(filters=256,kernel_size=5,activation='relu',strides= 1, kernel_initializer='he_uniform')(e)
pool = MaxPooling1D(pool_size=4)(conv1)
dropout= Dropout(0.5)(pool)
lstm1 = Bidirectional(LSTM(64, return_sequences=True))(dropout)
att=attention()(lstm1)
dense1 = Dense(units = 128, activation='relu', kernel_initializer='he_uniform')(att)
output= Dense(units = 1, activation = 'sigmoid', kernel_initializer= 'glorot_uniform')(dense1)
att_model1=keras.Model(seq_input,output)


In [ ]:
att_model1.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
attention_history=att_model1.fit(X_train, Y_train, epochs = 50, batch_size = 32, verbose = 1, validation_split = 0.1)

In [ ]:
score = att_model1.evaluate(X_test,Y_test,verbose = 1)
score[1]

# Hyperparameter Tuning

In [ ]:
def model_attention1_create(optimizer = 'Adam',
                 dropout_rate = 0.2, nb_filters = 256, kernel_size = 5,pool_size=2,units = 128,neurons = 20):
    
    seq_input = keras.Input(shape=(100,))
    e = Embedding(vocab_len, 100, input_length= 100,weights = [embedding_matrix], trainable= False)(seq_input)
    conv1 = Conv1D(nb_filters,kernel_size,activation='relu',strides= 1, kernel_initializer='he_uniform')(e)
    pool = MaxPooling1D(pool_size)(conv1)
    dropout= Dropout(0.5)(pool)
    lstm1 = Bidirectional(LSTM(10, return_sequences=True))(dropout)
    att=attention()(lstm1)
    dense1 = Dense(units = neurons, activation='relu', kernel_initializer='he_uniform')(att)
    output= Dense(units = 1, activation = 'sigmoid', kernel_initializer= 'glorot_uniform')(dense1)
    model=keras.Model(seq_input,output) 
    model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [ ]:
param_grid = {
    'nb_filters': [128,256],
    'kernel_size':[3,5],
    'pool_size':[2,4]

}                                                             ####perform grid search to find the hyperparameters

model = KerasClassifier(build_fn = model_attention1_create, verbose = 2)
grid_search = GridSearchCV(estimator = model, param_grid = param_grid, verbose = 2)
grid_results = grid_search.fit(X_train, Y_train, epochs = 10)
print('Best accuracy: ', grid_results.best_score_)
print('Best parameters: ', grid_results.best_params_)

In [ ]:
param_grid = {
    'neurons':[10,20,64,128]

}                                                             ####perform grid search to find the hyperparameters

model = KerasClassifier(build_fn = model_attention1_create, verbose = 2)
grid_search = GridSearchCV(estimator = model, param_grid = param_grid, verbose = 2)
grid_results = grid_search.fit(X_train, Y_train, epochs = 10)
print('Best accuracy: ', grid_results.best_score_)
print('Best parameters: ', grid_results.best_params_)

In [ ]:
param_grid = {
    'dropout_rate':[0.1,0.2,0.3,0.4,0.5]

}                                                             ####perform grid search to find the hyperparameters

model = KerasClassifier(build_fn = model_attention1_create, verbose = 2)
grid_search = GridSearchCV(estimator = model, param_grid = param_grid, verbose = 2)
grid_results = grid_search.fit(X_train, Y_train, epochs = 10)
print('Best accuracy: ', grid_results.best_score_)
print('Best parameters: ', grid_results.best_params_)

In [ ]:
param_grid = {
    'units': [10,20,64,128]
}                                                             ####perform grid search to find the hyperparameters

model = KerasClassifier(build_fn = model_attention1_create, verbose = 2)
grid_search = GridSearchCV(estimator = model, param_grid = param_grid, verbose = 2)
grid_results = grid_search.fit(X_train, Y_train, epochs = 10)
print('Best accuracy: ', grid_results.best_score_)
print('Best parameters: ', grid_results.best_params_)

# MODEL with Optimised Parameters

In [ ]:
seq_input = keras.Input(shape=(100,))
e = Embedding(vocab_len, 100, input_length= 100,weights = [embedding_matrix], trainable= False)(seq_input)
conv1 = Conv1D(filters=256,kernel_size=5,activation='relu',strides= 1, kernel_initializer='he_uniform')(e)
pool = MaxPooling1D(pool_size=4)(conv1)
dropout= Dropout(0.5)(pool)
lstm1 = Bidirectional(LSTM(20, return_sequences=True))(dropout)
att=attention()(lstm1)
dense1 = Dense(units = 128, activation='relu', kernel_initializer='he_uniform')(att)
output= Dense(units = 1, activation = 'sigmoid', kernel_initializer= 'glorot_uniform')(dense1)
att_model2=keras.Model(seq_input,output)

In [ ]:
att_model2.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
attention_history=att_model2.fit(X_train, Y_train, epochs = 50, batch_size = 32, verbose = 1, validation_split = 0.1)

In [ ]:
score = att_model2.evaluate(X_test,Y_test,verbose = 1)
score[1]

In [ ]:
import matplotlib.pyplot as plt                             ###plot loss on train and val data
plt.plot(attention_history.history['loss'])
plt.plot(attention_history.history['val_loss'])
plt.title('Model Loss Plot')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Train', 'Test'],loc = 'upper right')
plt.show()

In [ ]:
att_model2.save('attention_pretrained.hdf5')

In [ ]:
att_model2.load_weights('attention_pretrained.hdf5')

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, roc_curve, roc_auc_score
import matplotlib.pyplot as plt

In [ ]:
Y_pred = att_model2.predict(X_test)
Y_pred = (Y_pred >0.5)

In [ ]:
confusion = confusion_matrix(Y_test,Y_pred)        ###Model evaluation
precision = precision_score(Y_test,Y_pred)
recall = recall_score(Y_test,Y_pred)
F1 = f1_score(Y_test,Y_pred)
specificity = (confusion[0][0]/(confusion[0][0]+confusion[0][1]))
print('confusion matrix {}'.format(confusion))
print('precision is {}'.format(precision))
print('recall is  {}'.format(recall))
print('F1 score {}'.format(F1))
print('specificity is {}'.format(specificity))

In [ ]:
Y_pred_proba = att_model2.predict(X_test)
fpr, tpr, _ = roc_curve(Y_test, Y_pred_proba)
auc = roc_auc_score(Y_test, Y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()